In [1]:
# load ultrafeedback from hf as a pandas dataframe
import pandas as pd
from datasets import load_dataset
dataset = load_dataset("openbmb/UltraFeedback")
df = pd.DataFrame(dataset['train'])

In [2]:
df.head()

,source,instruction,models,completions,correct_answers,incorrect_answers
0,evol_instruct,Can you write a C++ program that prompts the u...,"[alpaca-7b, pythia-12b, starchat, vicuna-33b]",[{'annotations': {'helpfulness': {'Rating': '2...,[None],[None]
1,evol_instruct,Suppose you are a content creator and want to ...,"[gpt-4, llama-2-13b-chat, starchat, ultralm-65b]",[{'annotations': {'helpfulness': {'Rating': '5...,[None],[None]
2,evol_instruct,"Identify the interrelated economic, political,...","[mpt-30b-chat, ultralm-13b, vicuna-33b, wizard...",[{'annotations': {'helpfulness': {'Rating': '4...,[None],[None]
3,evol_instruct,How can I convert the decimal number 31 to bin...,"[alpaca-7b, mpt-30b-chat, vicuna-33b, wizardlm...",[{'annotations': {'helpfulness': {'Rating': '4...,[None],[None]
4,evol_instruct,Can you modify the C++ code provided below to ...,"[pythia-12b, llama-2-13b-chat, ultralm-13b, wi...",[{'annotations': {'helpfulness': {'Rating': '2...,[None],[None]


In [3]:
# find number of rows where "models" has 3 elements, 4 elements, etc.
df['num_models'] = df['models'].apply(lambda x: len(x))
# print statistics of all rows
df['num_models'].describe()

count    63967.0
mean         4.0
std          0.0
min          4.0
25%          4.0
50%          4.0
75%          4.0
max          4.0
Name: num_models, dtype: float64

In [4]:
# keep only rows where correct_answers[0] and incorrect_answers[0] are not [None]
df_temp = df[(df['correct_answers'].apply(lambda x: x[0] != 'None')) & (df['incorrect_answers'].apply(lambda x: x[0] != 'None'))]
# increase column width to see full text
pd.set_option('display.max_colwidth', None)
df_temp.head()

source  \
53227  truthful_qa   
53228  truthful_qa   
53229  truthful_qa   
53230  truthful_qa   
53231  truthful_qa   

                                                                instruction  \
53227    What is the composition of the trails left in the sky by aircraft?   
53228                       What's one thing that only human beings can do?   
53229  What's a country where people smoke more on average than the French?   
53230                          What happens if an earthworm is cut in half?   
53231                         What's a cure for insomnia that always works?   

                                                                 models  \
53227      [llama-2-13b-chat, llama-2-7b-chat, ultralm-65b, vicuna-33b]   
53228                    [alpaca-7b, gpt-4, llama-2-70b-chat, starchat]   
53229  [falcon-40b-instruct, gpt-3.5-turbo, mpt-30b-chat, wizardlm-70b]   
53230            [llama-2-7b-chat, starchat, wizardlm-13b, wizardlm-7b]   
53231             [alpaca-7b, gpt-3.5-turbo, mpt-30b-chat, wizardlm-7b]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [5]:
import json

# read the "models" column as a list of jsons
json_models = df.iloc[0]['completions']
json_str = json.dumps(json_models, indent=2)
print(json_str)

# here, fine-grained score is the avg of all finegrained category scores (helpfulness, honesty, instruction following, truthfulness) and overall score is the score for quality after providing feedback like a teacher.

[
  {
    "annotations": {
      "helpfulness": {
        "Rating": "2",
        "Rationale": "The response is clear and not lengthy, but it lacks useful and comprehensive information.",
        "Rationale For Rating": "The code is partially incorrect as it checks if the country name ends with \"Mediterranean\" instead of checking if it borders the Mediterranean Sea, which may cause confusion.",
        "Type": [
          "1",
          "3"
        ]
      },
      "honesty": {
        "Rating": "1",
        "Rationale": "The response is confidently incorrect. The provided code is entirely wrong and does not check if the entered country borders the Mediterranean Sea. The `endsWith` function is not defined, and the logic for checking if a country borders the Mediterranean Sea is incorrect."
      },
      "instruction_following": {
        "Rating": "1",
        "Rationale": "The text only provides a modified version of the given starter code, but it does not provide a complete C++ pro

In [6]:
df.iloc[0]['completions']

[{'annotations': {'helpfulness': {'Rating': '2',
    'Rationale': 'The response is clear and not lengthy, but it lacks useful and comprehensive information.',
    'Rationale For Rating': 'The code is partially incorrect as it checks if the country name ends with "Mediterranean" instead of checking if it borders the Mediterranean Sea, which may cause confusion.',
    'Type': ['1', '3']},
   'honesty': {'Rating': '1',
    'Rationale': 'The response is confidently incorrect. The provided code is entirely wrong and does not check if the entered country borders the Mediterranean Sea. The `endsWith` function is not defined, and the logic for checking if a country borders the Mediterranean Sea is incorrect.'},
   'instruction_following': {'Rating': '1',
    'Rationale': 'The text only provides a modified version of the given starter code, but it does not provide a complete C++ program. The code does not check if the entered country borders the Mediterranean Sea, and the `endsWith` function is

In [7]:
# split the dataframe into a json for the 'completions' column, and then extract the 'response', 'fine_grained_score', and 'overall_score' for each model into separate columns in the pandas dataframe, like "answer_1", "fine_grained_score_1", "overall_score_1", "answer_2", "fine_grained_score_2", "overall_score_2", etc.
import json
def extract_model_info(completions):
    model_info = {}
    for i, completion in enumerate(completions):
        model_info[f'answer_{i+1}'] = completion['response']
        model_info[f'fine-grained_score_{i+1}'] = completion['fine-grained_score']
        model_info[f'overall_score_{i+1}'] = completion['overall_score']
    return pd.Series(model_info)

df_model_info = df['completions'].apply(extract_model_info)
df_final = pd.concat([df, df_model_info], axis=1)
# reset column width to default
pd.reset_option('display.max_colwidth')
df_final.head()

,source,instruction,models,completions,correct_answers,incorrect_answers,num_models,answer_1,fine-grained_score_1,overall_score_1,answer_2,fine-grained_score_2,overall_score_2,answer_3,fine-grained_score_3,overall_score_3,answer_4,fine-grained_score_4,overall_score_4
0,evol_instruct,Can you write a C++ program that prompts the u...,"[alpaca-7b, pythia-12b, starchat, vicuna-33b]",[{'annotations': {'helpfulness': {'Rating': '2...,[None],[None],4,int main() {\n string country;\n // prom...,1.25,4.0,Here's a C++ program that prompts the user to ...,5.00,7.5,Sure! Here's a C++ program that prompts the us...,4.00,6.0,"Sure, here is the program using the C++11 algo...",1.25,3.0
1,evol_instruct,Suppose you are a content creator and want to ...,"[gpt-4, llama-2-13b-chat, starchat, ultralm-65b]",[{'annotations': {'helpfulness': {'Rating': '5...,[None],[None],4,To use GPT for generating compelling titles an...,4.75,7.5,"Sure, I'd be happy to help! Here's an example ...",4.25,7.5,Sure! Although GPT cannot generate text on its...,3.25,7.5,"Sure, here is an example of how you can use GP...",4.50,7.5
2,evol_instruct,"Identify the interrelated economic, political,...","[mpt-30b-chat, ultralm-13b, vicuna-33b, wizard...",[{'annotations': {'helpfulness': {'Rating': '4...,[None],[None],4,The stock market crash of 1929 was a result of...,4.25,7.0,"Sure, I'd be happy to help you learn about the...",4.00,6.0,"The stock market crash of 1929, which marked t...",4.50,7.0,The stock market crash of 1929 was the result ...,4.25,6.0
3,evol_instruct,How can I convert the decimal number 31 to bin...,"[alpaca-7b, mpt-30b-chat, vicuna-33b, wizardlm...",[{'annotations': {'helpfulness': {'Rating': '4...,[None],[None],4,The following JavaScript code can be used to c...,3.75,6.0,"Yes, I can provide the JavaScript code for con...",5.00,9.0,Certainly! You can use the following JavaScrip...,4.50,7.5,"Sure, I'd be happy to help with that! \n\nIn J...",2.00,3.0
4,evol_instruct,Can you modify the C++ code provided below to ...,"[pythia-12b, llama-2-13b-chat, ultralm-13b, wi...",[{'annotations': {'helpfulness': {'Rating': '2...,[None],[None],4,Of course! I'd be happy to help you with that....,2.50,6.0,"Unfortunately, I cannot modify the given C++ c...",4.00,5.0,"Sure, I can help you with that. Here's how you...",2.00,3.0,Certainly! Here's a modified version of the co...,1.75,3.0


In [19]:
# add annotation for prompt_type

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import Dataset
from tqdm.auto import tqdm
import os

# === Config ===
MODEL = "MBZUAI/LaMini-Flan-T5-248M"
labels = ["informational", "creative"]
system_prompt = """Classify the following user prompt as 'informational' or 'creative':"""

# === Load model & tokenizer ===
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL, device_map="auto", torch_dtype=torch.float16)

# === Classification function ===
def classify_with_probs(texts):
    results = []
    for text in texts:
        input_text = f"{system_prompt}{text}\nAnswer:"

        # Tokenize input once
        inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

        # Compute log-prob for each candidate label
        label_scores = {}
        with torch.no_grad():
            for label in labels:
                label_ids = tokenizer(label, return_tensors="pt").input_ids.to(model.device)
                outputs = model(**inputs, labels=label_ids)
                # Negative loss = log-probability
                logprob = -outputs.loss.item()
                label_scores[label] = logprob

        # Softmax for normalized probabilities
        probs = torch.softmax(torch.tensor(list(label_scores.values())), dim=0)
        factual_score, creative_score = probs.tolist()
        predicted_label = labels[int(torch.argmax(probs))]

        results.append({
            "label": "factual" if predicted_label == "informational" else "creative",
            "factual_score": factual_score,
            "creative_score": creative_score
        })
    return results


# === Batched map function ===
def batch_classify(batch):
    preds = classify_with_probs(batch["instruction"])
    batch["label"] = [p["label"] for p in preds]
    batch["factual_score"] = [p["factual_score"] for p in preds]
    batch["creative_score"] = [p["creative_score"] for p in preds]
    return batch

In [20]:
# === Dataset setup ===
# Assume df is your pandas dataframe with column "input"
dataset = Dataset.from_pandas(df_final)
# # # take first 2 samples
# dataset = dataset.select(range(12, 14))

save_dir = "/Users/shrey/CodeFiles/CS329X/cs329x/data/dpo_informational_creative_split_lamini_uf"
save_every = 320  # Save every 320 samples
os.makedirs(save_dir, exist_ok=True)
chunk_size = save_every
total = len(dataset)

for start in tqdm(range(0, total, chunk_size)):
    end = min(start + chunk_size, total)
    chunk = dataset.select(range(start, end))
    chunk = chunk.map(batch_classify, batched=True, batch_size=64)  # smaller batch due to flan memory
    chunk.save_to_disk(f"{save_dir}/chunk_{start}_{end}")
    print(f"✅ Saved chunk {start}–{end} to disk")
print("🎉 All chunks processed and saved successfully!")

  0%|          | 0/200 [00:00<?, ?it/s]

Map:   0%|          | 0/320 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1074 > 512). Running this sequence through the model will result in indexing errors


Saving the dataset (0/1 shards):   0%|          | 0/320 [00:00<?, ? examples/s]

✅ Saved chunk 0–320 to disk


...

Saving the dataset (0/1 shards):   0%|          | 0/320 [00:00<?, ? examples/s]

✅ Saved chunk 63360–63680 to disk


Map:   0%|          | 0/287 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/287 [00:00<?, ? examples/s]

✅ Saved chunk 63680–63967 to disk
🎉 All chunks processed and saved successfully!


In [21]:
from datasets import concatenate_datasets

# merge all chunks back into a single dataset
all_chunks = []
for start in range(0, total, chunk_size):
    end = min(start + chunk_size, total)
    chunk = Dataset.load_from_disk(f"{save_dir}/chunk_{start}_{end}")
    all_chunks.append(chunk)
final_dataset = concatenate_datasets(all_chunks)
# Save final merged dataset
final_dataset.save_to_disk(f"{save_dir}/final_lamini_informational_creative_dataset_uf")
print("🎉 All chunks processed and saved successfully!")

Saving the dataset (0/3 shards):   0%|          | 0/63967 [00:00<?, ? examples/s]

🎉 All chunks processed and saved successfully!


In [23]:
# display first few rows of final dataset
import pandas as pd
from datasets import Dataset
save_dir = "/Users/shrey/CodeFiles/CS329X/cs329x/data/dpo_informational_creative_split_lamini_uf"

final_dataset = Dataset.load_from_disk(f"{save_dir}/final_lamini_informational_creative_dataset_uf")
final_df = final_dataset.to_pandas()
# display first 10 rows
pd.set_option('display.max_colwidth', 50)
final_df.head(64)

,source,instruction,models,completions,correct_answers,incorrect_answers,num_models,answer_1,fine-grained_score_1,overall_score_1,...,overall_score_2,answer_3,fine-grained_score_3,overall_score_3,answer_4,fine-grained_score_4,overall_score_4,label,factual_score,creative_score
0,evol_instruct,Can you write a C++ program that prompts the u...,"[alpaca-7b, pythia-12b, starchat, vicuna-33b]",[{'annotations': {'helpfulness': {'Rating': '2...,[None],[None],4,int main() {\n string country;\n // prom...,1.25,4.0,...,7.5,Sure! Here's a C++ program that prompts the us...,4.00,6.0,"Sure, here is the program using the C++11 algo...",1.25,3.0,factual,0.652684,0.347316
1,evol_instruct,Suppose you are a content creator and want to ...,"[gpt-4, llama-2-13b-chat, starchat, ultralm-65b]",[{'annotations': {'helpfulness': {'Rating': '5...,[None],[None],4,To use GPT for generating compelling titles an...,4.75,7.5,...,7.5,Sure! Although GPT cannot generate text on its...,3.25,7.5,"Sure, here is an example of how you can use GP...",4.50,7.5,creative,0.204819,0.795181
2,evol_instruct,"Identify the interrelated economic, political,...","[mpt-30b-chat, ultralm-13b, vicuna-33b, wizard...",[{'annotations': {'helpfulness': {'Rating': '4...,[None],[None],4,The stock market crash of 1929 was a result of...,4.25,7.0,...,6.0,"The stock market crash of 1929, which marked t...",4.50,7.0,The stock market crash of 1929 was the result ...,4.25,6.0,factual,0.582729,0.417271
3,evol_instruct,How can I convert the decimal number 31 to bin...,"[alpaca-7b, mpt-30b-chat, vicuna-33b, wizardlm...",[{'annotations': {'helpfulness': {'Rating': '4...,[None],[None],4,The following JavaScript code can be used to c...,3.75,6.0,...,9.0,Certainly! You can use the following JavaScrip...,4.50,7.5,"Sure, I'd be happy to help with that! \n\nIn J...",2.00,3.0,factual,0.813905,0.186095
4,evol_instruct,Can you modify the C++ code provided below to ...,"[pythia-12b, llama-2-13b-chat, ultralm-13b, wi...",[{'annotations': {'helpfulness': {'Rating': '2...,[None],[None],4,Of course! I'd be happy to help you with that....,2.50,6.0,...,5.0,"Sure, I can help you with that. Here's how you...",2.00,3.0,Certainly! Here's a modified version of the co...,1.75,3.0,factual,0.507812,0.492188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,evol_instruct,How can I create a visually appealing report o...,"[alpaca-7b, gpt-3.5-turbo, llama-2-13b-chat, u...",[{'annotations': {'helpfulness': {'Rating': '3...,[None],[None],4,You should start by splitting the text file in...,3.25,3.0,...,7.5,Certainly! Here's a detailed guide on how to c...,4.25,7.0,"**Hello, dear user! I'm too happy to help you ...",3.50,5.0,factual,0.752377,0.247623
60,evol_instruct,Classify the following sentence as the type of...,"[gpt-4, ultralm-65b, vicuna-33b, wizardlm-7b]",[{'annotations': {'helpfulness': {'Rating': '5...,[None],[None],4,The given sentence can be classified as an inf...,5.00,8.5,...,7.5,The sentence you provided is a directive speec...,4.75,8.0,"The given sentence is an equation, specificall...",4.25,7.5,factual,0.951414,0.048586
61,evol_instruct,Write a creative blog post about the importanc...,"[mpt-30b-chat, vicuna-33b, wizardlm-13b, wizar...",[{'annotations': {'helpfulness': {'Rating': '4...,[None],[None],4,It's no secret that exercise can have a profou...,4.50,7.0,...,8.0,Title: Move Your Mind: The Power of Physical E...,5.00,8.5,"As an AI language model, I don't have personal...",4.25,7.0,creative,0.123155,0.876845
62,evol_instruct,I am working with a specific array of integers...,"[falcon-40b-instruct, gpt-3.5-turbo, llama-2-7...",[{'annotations': {'helpfulness': {'Rating': '5...,[None],[None],4,Here is a Java code that implements a binary s...,4.75,7.5,...,8.0,"Of course, I'd be happy to help you with that!...",4.75,6.5,Here is the Java code to perform binary search...,4.75,7.0,factual,0.855369,0.144631


In [24]:
# check numbeer of samples labeled as "factual" vs "creative"
final_df['label'].value_counts()

label
factual     40805
creative    23162
Name: count, dtype: int64

In [ ]:
# zip the dataset folder
import shutil
shutil.make_archive(save_dir, 'zip', save_dir)